# Preprocess Population Dataset
We preprocess data for training.The population dataset will be considered:

In [1]:
import pandas as pd
import os.path as op
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = '../../Data'

## Noisy Data :

In [3]:
population_claims = pd.read_csv(op.join(DATA_DIR, 'population/Unprocessed_data/population_claims.csv'))

In [4]:
population_claims.head()

ObjectID      PropertyID  PropertyValue                  SourceID  \
0  abu dhabi  Population2006      1000230.0         0 (68.162.248.83)   
1  abu dhabi  Population2006      1850230.0  1513217: Mohammedfairouz   
2  amsterdam  Population2006       741329.0             141597: Ilse@   
3  amsterdam  Population2006       742884.0           1300620: Krator   
4   adelaide  Population2006      1124315.0         3922171: Pirate05   

   TimeStamp  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [5]:
population_claims.shape

(49943, 5)

In [6]:
claims = population_claims.copy()

## Convert the column of PropertyValue into integer 

In [7]:
claims.PropertyValue=claims.PropertyValue.astype("int64")

In [8]:
claims.dtypes

ObjectID          object
PropertyID        object
PropertyValue      int64
SourceID          object
TimeStamp        float64
dtype: object

## Null values 

In [9]:
claims.isnull().sum()

ObjectID             0
PropertyID           0
PropertyValue        0
SourceID             0
TimeStamp        49943
dtype: int64

## Drop duplicates

In [10]:
claims.drop_duplicates().shape, claims.shape

((49943, 5), (49943, 5))

## True data

In [11]:
population_truths = pd.read_csv(op.join(DATA_DIR, 'population/Unprocessed_data/population_truth.csv'))

In [12]:
population_truths.head()

ObjectID      PropertyID  PropertyValue
0      cleveland, ohio  Population2000         478403
1        gary, indiana  Population2000         102746
2      flint, michigan  Population2000         124943
3  compton, california  Population2000          93493
4     washington, d.c.  Population2005         582049

In [13]:
population_truths.shape

(308, 3)

In [14]:
truths = population_truths.copy()

In [15]:
truths.dtypes

ObjectID         object
PropertyID       object
PropertyValue     int64
dtype: object

## Instead of Having "ObjectID" and "PropertyID", we join them to create one column called   "Object" and we also drop the column "TimeStamp" 

In [16]:
claims_DL=claims.copy()
truths_DL=truths.copy()

In [17]:
claims_DL['object'] = claims_DL.ObjectID.apply(lambda x: x.replace(',', '_').replace(' ', '')) + '_' + claims_DL.PropertyID
claims_DL['value'] = claims_DL.PropertyValue
claims_DL.drop(columns=['ObjectID', 'PropertyID', 'PropertyValue', 'TimeStamp'], inplace=True)

In [18]:
truths_DL['object'] = truths_DL.ObjectID.apply(lambda x: x.replace(',', '_').replace(' ', '')) + '_' + truths_DL.PropertyID
truths_DL['value'] = truths_DL.PropertyValue
truths_DL.drop(columns=['ObjectID', 'PropertyID', 'PropertyValue'], inplace=True)

In [19]:
claims_DL = pd.merge(claims_DL, truths_DL, on=['object','value'], how='left', indicator='label')
claims_DL ['label'] = np.where(claims_DL.label == 'both', 0, 1)
print (claims_DL)

                        SourceID  \
0              0 (68.162.248.83)   
1       1513217: Mohammedfairouz   
2                  141597: Ilse@   
3                1300620: Krator   
4              3922171: Pirate05   
...                          ...   
49938            5277842: Kotbot   
49939            5277842: Kotbot   
49940            5277842: Kotbot   
49941  6122788: Weatherman Tyler   
49942         6126311: Hauschild   

                                                 object    value  label  
0                               abudhabi_Population2006  1000230      1  
1                               abudhabi_Population2006  1850230      1  
2                              amsterdam_Population2006   741329      1  
3                              amsterdam_Population2006   742884      1  
4                               adelaide_Population2006  1124315      1  
...                                                 ...      ...    ...  
49938                     gminakoniecpol_Populati

In [20]:
claims_DL= pd.DataFrame(claims_DL)

In [21]:
claims_DL.label.value_counts()

1    49352
0      591
Name: label, dtype: int64

In [26]:
claims_DL

SourceID  \
0              0 (68.162.248.83)   
1       1513217: Mohammedfairouz   
2                  141597: Ilse@   
3                1300620: Krator   
4              3922171: Pirate05   
...                          ...   
49938            5277842: Kotbot   
49939            5277842: Kotbot   
49940            5277842: Kotbot   
49941  6122788: Weatherman Tyler   
49942         6126311: Hauschild   

                                                 object    value  label  
0                               abudhabi_Population2006  1000230      1  
1                               abudhabi_Population2006  1850230      1  
2                              amsterdam_Population2006   741329      1  
3                              amsterdam_Population2006   742884      1  
4                               adelaide_Population2006  1124315      1  
...                                                 ...      ...    ...  
49938                     gminakoniecpol_Population2006    10300      1  
49939                     gminakonopiska_Population2006    10405      1  
49940                      gminakruszyna_Population2006     4899      1  
49941  bristol_connecticutfiredepartment_Population2005    61353      1  
49942                  glidden_wisconsin_Population2000     1253      1  

[49943 rows x 4 columns]

## Save the Data in files

In [22]:
truths.to_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/truths_DL_1.csv'), index=False)

In [23]:
claims.to_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/claims_DL_1.csv'), index=False)

In [24]:
truths.shape, claims.shape

((308, 3), (49943, 5))

In [25]:
claims_DL.to_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/claims_DL_2.csv'), index=False)